In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df=pd.read_csv('labels.csv')
df

,filepath,xmin,xmax,ymin,ymax
0,./plates/images\1.xml,110,193,123,139
1,./plates/images\10.p.xml,149,274,573,631
2,./plates/images\10.xml,143,246,358,389
3,./plates/images\1000.xml,528,627,631,660
4,./plates/images\11.p.xml,336,515,654,687
...,...,...,...,...,...
620,./plates/images\Screenshot from 2023-05-27 13-...,296,396,511,533
621,./plates/images\Screenshot from 2023-05-27 13-...,327,459,656,690
622,./plates/images\Screenshot from 2023-05-27 13-...,256,481,567,608
623,./plates/images\Screenshot from 2023-05-27 13-...,258,478,575,629


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename=df['filepath'][0]
filename

'./plates/images\\1.xml'

In [5]:
def getFilename(filename):
   filename_image=xet.parse(filename).getroot().find('filename').text
   filepath_image=os.path.join('./plates/images',filename_image)
   return filepath_image

In [6]:
getFilename(filename)

'./plates/images\\1.jpeg'

In [7]:
image_path=list(df['filepath'].apply(getFilename))
image_path

['./plates/images\\1.jpeg',
 './plates/images\\10.p.png',
 './plates/images\\10.jpg',
 './plates/images\\1000.png',
 './plates/images\\11.p.png',
 './plates/images\\11.jpeg',
 './plates/images\\12.p.png',
 './plates/images\\12.jpeg',
 './plates/images\\13.p.png',
 './plates/images\\13.jpeg',
 './plates/images\\15.png',
 './plates/images\\17.png',
 './plates/images\\18.png',
 './plates/images\\19.png',
 './plates/images\\2.p.png',
 './plates/images\\2.jpeg',
 './plates/images\\20.png',
 './plates/images\\200.png',
 './plates/images\\201.png',
 './plates/images\\202.png',
 './plates/images\\203.png',
 './plates/images\\205.png',
 './plates/images\\207.png',
 './plates/images\\208.png',
 './plates/images\\21.png',
 './plates/images\\210.png',
 './plates/images\\211.png',
 './plates/images\\212.png',
 './plates/images\\213.png',
 './plates/images\\214.png',
 './plates/images\\216.png',
 './plates/images\\217.png',
 './plates/images\\218.png',
 './plates/images\\22.png',
 './plates/images\\

In [8]:
#verify image and output



In [9]:
file_path=image_path[0]
file_path


'./plates/images\\1.jpeg'

In [10]:
img=cv2.imread(file_path)

#Able to resize it 
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
cv2.rectangle(img,(110,123),(193,138),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
# Data Preprocessing


In [13]:

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [14]:
labels=df.iloc[:,1:].values
len(image_path)

625

In [15]:
data = []
output = []

for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)

   

    h, w, d = img_arr.shape

    # Preprocessing using load_img function
    load_image = load_img(image, target_size=(224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr / 255.0  # Normalization

    # Normalization of labels
    xmin, xmax, ymin, ymax = labels[ind]

    # Normalize x by width and y by height
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    label_norm = (nxmin, nxmax, nymin, nymax)  # Normalized output

    # Append to data and output lists
    data.append(norm_load_image_arr)
    output.append(label_norm)


In [16]:
x=np.array(data,dtype=np.float32)
y=np.array(output,dtype=np.float32)

In [17]:
x.shape,y.shape

((625, 224, 224, 3), (625, 4))

In [18]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((500, 224, 224, 3), (125, 224, 224, 3), (500, 4), (125, 4))

In [19]:
# Deep Learning Model

In [20]:
from tensorflow.keras.applications import MobileNetV2,InceptionV3,InceptionResNetV2
from tensorflow.keras.layers import Dense,Dropout,Flatten,Input
from tensorflow.keras.models import Model 
import tensorflow as tf



In [21]:
inception_resnet=InceptionResNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False #Not changing the weight of the image 

#----------------------------------------------------------
#Adding layer in the model or making a neural network

headmodel=inception_resnet.output
headmodel=Flatten()(headmodel)
headmodel=Dense(500,activation="relu")(headmodel)
headmodel=Dense(250,activation="relu")(headmodel)


headmodel=Dense(4,activation='sigmoid')(headmodel)

#-----------------------------model
model=Model(inputs=inception_resnet.input,outputs=headmodel)

In [22]:
#Compile the model
#Defining learning rate and loss function
# Compile the model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

# Print model summary
model.summary()


#so this  

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 25, 25, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 25, 25, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 25, 25, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 25, 25, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_21[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 25, 25, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 25, 25, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 25, 25, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 25, 25, 64)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 25, 25, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 25, 25, 32)   0           ['batch_normalization_37[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 25, 25, 48)   0           ['batch_normalization_40[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_169[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_167 (Batch  (None, 5, 5, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_175 (Batch  (None, 5, 5, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_178 (Batch  (None, 5, 5, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_175[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_178[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_186[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 custom_scale_layer_35 (CustomS  (None, 5, 5, 2080)  0           ['block8_5_ac[0][0]',            
 caleLayer)                                                       'block8_6_conv[0][0]']          
          

                                                                                                  
 custom_scale_layer_37 (CustomS  (None, 5, 5, 2080)  0           ['block8_7_ac[0][0]',            
 caleLayer)                                                       'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['custom_scale_layer_37[0][0]']  
                                                                                                  
 conv2d_196 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 5, 5, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten (Flatten)              (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 500)          19200500    ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 250)          125250      ['dense[0][0]']                  
                                                                                                  
 dense_2 (

In [23]:
# Training the model

In [24]:
from tensorflow.keras.callbacks import TensorBoard

In [25]:
tfb=TensorBoard('object_detection')

In [26]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=100, validation_data=(x_test, y_test), callbacks=[tfb])

Epoch 1/100
50/50 [==============================] - 152s 3s/step - loss: 0.1216 - val_loss: 0.1090
Epoch 2/100
50/50 [==============================] - 85s 2s/step - loss: 0.1159 - val_loss: 0.1265
Epoch 3/100
50/50 [==============================] - 100s 2s/step - loss: 0.1097 - val_loss: 0.1028
Epoch 4/100
50/50 [==============================] - 96s 2s/step - loss: 0.1067 - val_loss: 0.1090
Epoch 5/100
50/50 [==============================] - 100s 2s/step - loss: 0.1065 - val_loss: 0.1031
Epoch 6/100
50/50 [==============================] - 91s 2s/step - loss: 0.1077 - val_loss: 0.1139
Epoch 7/100
50/50 [==============================] - 89s 2s/step - loss: 0.1027 - val_loss: 0.1002
Epoch 8/100
50/50 [==============================] - 83s 2s/step - loss: 0.0989 - val_loss: 0.0971
Epoch 9/100
50/50 [==============================] - 86s 2s/step - loss: 0.0986 - val_loss: 0.0977
Epoch 10/100
50/50 [==============================] - 100s 2s/step - loss: 0.0971 - val_loss: 0.1005
Epoch

Epoch 83/100
50/50 [==============================] - 101s 2s/step - loss: 0.0013 - val_loss: 0.0080
Epoch 84/100
50/50 [==============================] - 87s 2s/step - loss: 0.0013 - val_loss: 0.0080
Epoch 85/100
50/50 [==============================] - 81s 2s/step - loss: 0.0012 - val_loss: 0.0079
Epoch 86/100
50/50 [==============================] - 91s 2s/step - loss: 0.0012 - val_loss: 0.0082
Epoch 87/100
50/50 [==============================] - 88s 2s/step - loss: 0.0014 - val_loss: 0.0081
Epoch 88/100
50/50 [==============================] - 82s 2s/step - loss: 0.0012 - val_loss: 0.0085
Epoch 89/100
50/50 [==============================] - 94s 2s/step - loss: 0.0014 - val_loss: 0.0080
Epoch 90/100
50/50 [==============================] - 93s 2s/step - loss: 0.0014 - val_loss: 0.0082
Epoch 91/100
50/50 [==============================] - 91s 2s/step - loss: 0.0012 - val_loss: 0.0084
Epoch 92/100
50/50 [==============================] - 86s 2s/step - loss: 0.0013 - val_loss: 0.0082

In [27]:
history = model.fit(x=x_train, y=y_train, batch_size=10, epochs=200, validation_data=(x_test, y_test), callbacks=[tfb],initial_epoch=101)


Epoch 102/200
50/50 [==============================] - 87s 2s/step - loss: 0.0012 - val_loss: 0.0085
Epoch 103/200
50/50 [==============================] - 81s 2s/step - loss: 0.0015 - val_loss: 0.0083
Epoch 104/200
50/50 [==============================] - 82s 2s/step - loss: 0.0013 - val_loss: 0.0080
Epoch 105/200
50/50 [==============================] - 93s 2s/step - loss: 0.0012 - val_loss: 0.0083
Epoch 106/200
50/50 [==============================] - 90s 2s/step - loss: 0.0012 - val_loss: 0.0083
Epoch 107/200
50/50 [==============================] - 91s 2s/step - loss: 0.0014 - val_loss: 0.0090
Epoch 108/200
50/50 [==============================] - 95s 2s/step - loss: 0.0014 - val_loss: 0.0084
Epoch 109/200
50/50 [==============================] - 93s 2s/step - loss: 0.0013 - val_loss: 0.0078
Epoch 110/200
50/50 [==============================] - 100s 2s/step - loss: 0.0012 - val_loss: 0.0081
Epoch 111/200
50/50 [==============================] - 94s 2s/step - loss: 0.0013 - val_lo

Epoch 183/200
50/50 [==============================] - 80s 2s/step - loss: 0.0011 - val_loss: 0.0085
Epoch 184/200
50/50 [==============================] - 91s 2s/step - loss: 0.0012 - val_loss: 0.0083
Epoch 185/200
50/50 [==============================] - 95s 2s/step - loss: 0.0011 - val_loss: 0.0081
Epoch 186/200
50/50 [==============================] - 94s 2s/step - loss: 0.0011 - val_loss: 0.0082
Epoch 187/200
50/50 [==============================] - 84s 2s/step - loss: 0.0011 - val_loss: 0.0081
Epoch 188/200
50/50 [==============================] - 80s 2s/step - loss: 0.0012 - val_loss: 0.0084
Epoch 189/200
50/50 [==============================] - 81s 2s/step - loss: 0.0013 - val_loss: 0.0081
Epoch 190/200
50/50 [==============================] - 84s 2s/step - loss: 0.0013 - val_loss: 0.0084
Epoch 191/200
50/50 [==============================] - 84s 2s/step - loss: 0.0012 - val_loss: 0.0084
Epoch 192/200
50/50 [==============================] - 89s 2s/step - loss: 0.0012 - val_los

In [28]:
model.save('./models/object_detection.h5')